# Лабораторная работа №3

[файл с заданиями](https://github.com/ar1st0crat/PythonCourse/blob/master/labs/Lab03%20-%20Python%20OOP%20and%20GUI.pdf)

In [1]:
from IPython.display import Image
from abc import ABC, abstractmethod
from PyQt5.QtWidgets import QMainWindow, QApplication, QListView, QMenuBar, QLabel, QFileDialog, QMessageBox
from PyQt5.QtCore import QStringListModel
from enum import Enum, unique
from string_formatter import Ui_StringFormatter
import os
import sys
import re


1. Задан простой класс `Fraction` для представления дробей:
```python
class Fraction(object):
    def __init__(self, num, den):
        self.__num = num
        self.__den = den
        self.reduce()
    
    def __str__(self):
        return "%d/%d" % (self.__num, self.__den)
    
    def reduce(self):
        g = Fraction.gcd(self.__num, self.__den)
        self.__num /= g
        self.__den /= g
    
    @staticmethod
    def gcd(n, m):
        if m == 0:
            return n
        else:
            return Fraction.gcd(m, n % m)
```
Дополнить класс таким образом, чтобы выполнялся следующий код:
```python
frac = Fraction(7, 2)
print(-frac)             # выводит -7/2
print(~frac)             # выводит 2/7
print(frac**2)           # выводит 49/4
print(float(frac))       # выводит 3.5
print(int(frac))         # выводит 3```

In [13]:
class Fraction(object):
    def __init__(self, num, den):
        self.__num = num
        self.__den = den
        self.reduce()

    def __str__(self):
        return "%d/%d" % (self.__num, self.__den)

    def reduce(self):
        g = Fraction.gcd(self.__num, self.__den)
        self.__num /= g
        self.__den /= g

    @staticmethod
    def gcd(n, m):
        if m == 0:
            return n
        else:
            return Fraction.gcd(m, n % m)

    def __neg__(self):
        return Fraction(-self.__num, self.__den)

    def __invert__(self):
        return Fraction(self.__den, self.__num)

    def __pow__(self, power, modulo=None):
        return Fraction(self.__num ** power, self.__den ** power)

    def __int__(self):
        return int(self.__num // self.__den)

    def __float__(self):
        return self.__num / self.__den


In [8]:
#Работа дополненного класса
frac = Fraction(7, 2)
print(-frac)             # выводит -7/2
print(~frac)             # выводит 2/7
print(frac**2)           # выводит 49/4
print(float(frac))       # выводит 3.5
print(int(frac))         # выводит 3

-7/2
2/7
49/4
3.5
3


2. Напишите классы «Книга» (с обязательными полями: название, автор, код), «Библиотека» (с обязательными полями: адрес, номер) и корректно свяжите их. Код книги должен назначаться автоматически при добавлении книги в библиотеку (используйте для этого статический член класса). Если в конструкторе книги указывается в параметре пустое название, необходимо сгенерировать исключение (например, `ValueError`). Книга должна реализовывать интерфейс `Taggable` с методом `tag()`, который создает на основе строки набор тегов (разбивает строку на слова и возвращает только те, которые начинаются с большой буквы). Например, `tag()` для книги с названием _‘War and Peace’_  вернет список тегов `[‘War’, ‘Peace’]`. Реализуйте классы таким образом, чтобы корректно выполнялся следующий код:

```python
lib = Library(1, ’51 Some str., NY’)
lib += Book(‘Leo Tolstoi’, ‘War and Peace’)
lib += Book(‘Charles Dickens’, ‘David Copperfield’)

for book in lib:
    # вывод в виде: [1] L.Tolstoi ‘War and Peace’
    print(book)    
    # вывод в виде: [‘War’, ‘Peace’]
    print(book.tag())```

In [2]:
class Taggeble(ABC):
    @abstractmethod
    def tag(self):
        pass

class Book(Taggeble):
    __slots__ = ['__name', '__author', '__code']
    
    def __init__(self, author: str, name: str):
        if author and name:
            self.__name = name
            self.__author = author
            self.__code = None
        else:
            raise ValueError('Empty!')
    
    def tag(self):
        return list(filter(lambda x: x[0].isupper(), self.__name.split()))
    
    def __repr__(self):
        if self.__code:
            return "[{}] {}.{} '{}'".format(
                self.__code, self.__author[0], self.__author.split()[1], self.__name)
        else:
            return "{}.{} '{}'".format(
                self.__author[0], self.__author.split()[1], self.__name)
    
    @property
    def code(self):
        return self.__code
    
    @code.setter
    def code(self, code_):
        self.__code = code_
        
class Library(object):
    __slots__ = ['__number', '__address', '__books']
    __last_add_code = 0
    
    def __init__(self, number, address):
        self.__number = number
        self.__address = address
        self.__books = list()
    
    def __iadd__(self, other: Book):
        Library.__last_add_code += 1
        other.code = Library.__last_add_code
        self.__books.append(other)
        return self
    
    def __iter__(self):
        for book in self.__books:
            yield book


In [12]:
lib = Library(1, '51 Some str., NY')
lib += Book('Leo Tolstoi', 'War and Peace')
lib += Book('Charles Dickens', 'David Copperfield')
for book in lib:
    # вывод в виде: [1] L.Tolstoi ‘War and Peace’
    print(book)
    # вывод в виде: [‘War’, ‘Peace’]
    print(book.tag())


[1] L.Tolstoi 'War and Peace'
['War', 'Peace']
[2] C.Dickens 'David Copperfield'
['David', 'Copperfield']


3. Создайте графическую оболочку для скрипта, написанного в ходе выполнения задания № 4 лабораторной работы № 2, в виде диалогового окна. Рекомендуется использовать wxPython или PyQt.

![требования к окну](pic_1.jpg)

Требования к окну и скрипту:
- всю область окна должен занимать список с результатами поиска строк по шаблону в файле и указанием даты и времени поиска. Поиск производится автоматически при каждом открытии какого-либо файла, при этом список не очищается, а пополняется новыми результатами. При запуске скрита список изначально должен быть пустым (из файла лога данные подгружать не нужно);
- строка меню содержит пункты _«Файл»_ (с подпунктом _«Открыть...»_ для открытия файла, в котором необходимо искать строки) и _«Лог»_ (с подпунктами _«Экспорт...»_, _«Добавить в лог»_, _«Просмотр»_). Файл лога находится в рабочей папке скрипта и называется `script18.log`. Если файл отсутствует, скрипт при запуске должен выдать диалоговое окно с информацией «Файл лога не найден. Файл будет создан автоматически» и кнопкой `«ОК»`.
    - При выборе пункта меню _«Экспорт...»_ содержимое списка должно сохраниться в файле, который укажет пользователь. 
    - При выборе пункта _«Добавить в лог»_ содержимое списка приписывается в конец файла `script18.log`. 
    - При выборе пункта _«Просмотр»_ текущее содержимое списка удаляется, и список заполняется данными из лога. Перед этим действием скрипт должен выдать диалоговое окно с вопросом _«Вы действительно хотите открыть лог? Данные последних поисков будут потеряны!»_ и кнопками `«Да»` и `«Нет»`;    
- статусная строка должна состоять из двух полей: 
    - в первом поле (60% ширины окна), в зависимости от последнего произведенного действия, выводится либо текст *«Открыт лог»*, либо текст *«Обработан файл <полное_имя_файла>»*; 
    - второе поле (40% ширины окна) служит для отображения размера последнего обработанного файла в байтах. Эта строка форматируется: выводятся пробелы между степенями тысячи (например, *«2 036 231 байт»*);
- файлы нужно открывать и сохранять с помощью стандартного диалогового окна.

In [3]:
class FounderWin(QMainWindow):
    log = 'script18.log'

    def __init__(self):
        super().__init__()
        self.setWindowTitle('Искатель строк')
        self.list_model = QStringListModel()
        self.setCentralWidget(QListView())
        self.centralWidget().setModel(self.list_model)
        self.setGeometry(400, 300, 400, 300)
        self.setMenuBar(self.menu())
        self.status = QLabel('')
        self.statusBar().addWidget(self.status, self.width() * 0.6)
        self.last_size_status = QLabel('')
        self.statusBar().addPermanentWidget(self.last_size_status, self.width() * 0.4)
        self.is_log()

        self.show()

    def is_log(self):
        if not os.path.exists(os.path.abspath(self.log)):
            log_text = 'Файл лога не найден. Файл будет создан автоматически'
            QMessageBox.warning(self, 'Внимание', log_text)
            with open(self.log, 'w'):
                pass
            # m.exec_()

    def write_to_file(self, file, mode):
        with open(file, mode) as f:
            for sl in self.list_model.stringList():
                f.write(sl + '\n')

    def menu(self):
        menu_bar = QMenuBar()
        m = menu_bar.addMenu('Файл')
        a = m.addAction('Открыть...')
        a.triggered.connect(self.open_file)
        m = menu_bar.addMenu('Лог')
        a = m.addAction('Экспорт...')
        a.triggered.connect(self.export_log)
        a = m.addAction('Добавить в лог')
        a.triggered.connect(self.add_to_log)
        a = m.addAction('Просмотр')
        a.triggered.connect(self.view_log)
        return menu_bar

    def open_file(self):
        d = QFileDialog().getOpenFileName(self, 'Open file', sys.path[0])[0]
        if d:
            self.find_time(d)
            self.status.setText('Обработан файл ' + d)
            self.last_size_status.setText(self.last_file_size(d))

    @staticmethod
    def last_file_size(file):
        size = str(os.path.getsize(file))
        l = len(size)
        sl = [size[l - 3 * i - 3:l - 3 * i] for i in range(l // 3, -1, -1)]
        return size[0: l % 3] + ' '.join(sl) + ' байт'

    def export_log(self):
        d = QFileDialog().getOpenFileName(self, 'Open file', sys.path[0])[0]
        if d:
            self.write_to_file(d, 'w')
        pass

    def add_to_log(self):
        self.write_to_file(self.log, 'a')
        pass

    def view_log(self):
        text_msq = 'Вы действительно хотите открыть лог? Данные последних поисков будут потеряны!'
        msq = QMessageBox.question(self, 'Внимание', text_msq, buttons=QMessageBox.Yes | QMessageBox.No)
        if msq == QMessageBox.Yes:
            with open(self.log, 'r') as f:
                self.list_model.setStringList(f.read().split('\n'))
            self.status.setText('Открыт лог')
        pass

    def find_time(self, file):
        try:
            with open(file, 'r') as f:
                text = f.read().split('\n')
        except Exception as e:
            try:
                with open(file, 'r', encoding='utf8') as f:
                    text = f.read().split('\n')
            except Exception as e:
                find = ['Файл ' + file, ' ', 'Не удалось прочитать файл..', ' ']
        if text:
            find = ['Файл ' + file, ''] + ["Строка {}, позиция {} : найдено '{}'".format(j, m.start(), m.group(0))
                                           for
                                           j, t in enumerate(text) for m in
                                           re.finditer(r"(([0-1]\d|[2][0-3]):([0-5]\d):([0-5]\d))", t)]
            if find[-1] == '':
                find.append('Ничего не найдено..')
            find.append('')
        self.list_model.setStringList(self.list_model.stringList() + find)

In [4]:
app = QApplication(sys.argv)
ex = FounderWin()
try:
    sys.exit(app.exec_())
except SystemExit as se:
    print('Program has exited with code {}'.format(se))


Program has exited with code 0


4. Напишите простой класс `StringFormatter`для форматирования строк со следующим функционалом:
    - удаление всех слов из строки, длина которых меньше n букв;
    - замена всех цифр в строке на знак «*»;
    - вставка по одному пробелу между всеми символами в строке;
    - сортировка слов по размеру;
    - сортировка слов в лексикографическом порядке.
    
**Примечание**. *Разделители слов можно задавать отдельно. По
умолчанию в качестве разделителя принимается только символ
пробела.*

In [2]:
class StringFormatter(object):
    @unique
    class Sorting(Enum):
        NoSort = 0
        ByLength = 1
        ByAlphabet = 2

    __slots__ = ['__string', '__min_length', '__replace', '__rep_symbol', '__is_space', '__sort']

    def __init__(self, min_length=None, replace=False, rep_symbol='*', is_space=False, sort=Sorting.NoSort):
        self.__min_length = min_length
        self.__rep_symbol = rep_symbol
        self.__is_space = is_space
        self.__sort = sort
        self.__replace = replace
        pass

    def setSort(self, sort: Sorting):
        self.__sort = sort

    def setReplace(self, rep: bool):
        self.__replace = rep

    def setReplaceSymdol(self, sym: str):
        self.__rep_symbol = sym

    def setMinLength(self, len: int):
        self.__min_length = len

    def setSpace(self, is_space: bool):
        self.__is_space = is_space

    def __filter__(self, string_list):
        if self.__min_length:
            return list(filter(lambda x: len(x) >= self.__min_length, string_list))
        return string_list

    def __replace__(self, string_list):
        if self.__replace:
            return [i.translate(str.maketrans('0123456789', self.__rep_symbol * 10)) for i in string_list]
        return string_list

    def __space__(self, string_list):
        if self.__is_space:
            return [' '.join([s for s in sl]) for sl in string_list]
        return string_list

    def __sort__(self, string_list: list):
        if self.__sort == StringFormatter.Sorting.NoSort:
            return string_list
        if self.__sort == StringFormatter.Sorting.ByAlphabet:
            return sorted(string_list, key=str.lower)
        if self.__sort == StringFormatter.Sorting.ByLength:
            return sorted(string_list, key=len)

    def format(self, string: str) -> str:
        text = string.split()
        text = self.__filter__(text)
        text = self.__replace__(text)
        text = self.__sort__(text)
        text = self.__space__(text)
        return ' '.join(text)

In [7]:
formatter = StringFormatter(min_length=5, sort=StringFormatter.Sorting.ByLength)
st = 'abcdef bbbbbbb123456 hui hiub aaaaaaaa'
print(formatter.format(st))


abcdef aaaaaaaa bbbbbbb123456


5. Напишите скрипт с графическим интерфейсом пользователя для демонстрации работы класса `StringFormatter`. Примеры окон приведены на рисунке (все элементы управления необходимо обязательно реализовать те же, что присутствуют на рисунке). Разные комбинации отмеченных чекбоксов приводят к разным цепочкам операций форматирования задаваемой в верхнем поле строки с разными результатами:

![работа скрипта 1](pic_2.jpg) ![работа скрипта 2](pic_3.jpg)

In [3]:
class StringFormatterWin(QMainWindow):
    def __init__(self, parent=None):
        super(StringFormatterWin, self).__init__(parent)
        self.ui = Ui_StringFormatter()
        self.ui.setupUi(self)
        self.ui.spinBox.setEnabled(False)
        self.ui.line_rep_symbol.setEnabled(False)
        self.ui.widget.setEnabled(False)
        self.ui.radioButton.setChecked(True)
        self.ui.line_rep_symbol.setText('*')
        self.ui.check_min_length.clicked.connect(lambda x: self.ui.spinBox.setEnabled(not self.ui.spinBox.isEnabled()))
        self.ui.check_rep.clicked.connect(
            lambda x: self.ui.line_rep_symbol.setEnabled(not self.ui.line_rep_symbol.isEnabled()))
        self.ui.check_sort.clicked.connect(lambda x: self.ui.widget.setEnabled(not self.ui.widget.isEnabled()))
        self.ui.pushButton.clicked.connect(self.__format)

        self.show()

    def __format(self):
        formatter = StringFormatter()
        if self.ui.check_min_length.isChecked():
            formatter.setMinLength(self.ui.spinBox.value())
        if self.ui.check_rep.isChecked():
            formatter.setReplace(True)
            if self.ui.line_rep_symbol.text():
                formatter.setReplaceSymdol(self.ui.line_rep_symbol.text())
        if self.ui.check_space.isChecked():
            formatter.setSpace(True)
        if self.ui.check_sort.isChecked():
            if self.ui.radioButton.isChecked():
                formatter.setSort(StringFormatter.Sorting.ByLength)
            else:
                formatter.setSort(StringFormatter.Sorting.ByAlphabet)
        self.ui.line_output.setText(formatter.format(self.ui.line_input.text()))


In [4]:
app = QApplication(sys.argv)
ex = StringFormatterWin()
try:
    sys.exit(app.exec_())
except SystemExit as se:
    print('Program has exited with code {}'.format(se))

Program has exited with code 0
